In [28]:
using Pkg
Pkg.activate("rad_transfer"; shared=true)
using RadiativeTransfer
using RadiativeTransfer.CrossSection
using KernelAbstractions
using ..Architectures
using Pkg.Artifacts
using ProgressMeter
using DataStructures

include("julia_cs_matrix_helper.jl")
# include("/home/cchristo/proj_christian/RadiativeTransfer.jl/src/CrossSection/types.jl")


 Activating environment at `~/.julia/environments/rad_transfer/Project.toml`
 Activating environment at `~/.julia/environments/rad_transfer/Project.toml`


get_cross_section_matrix (generic function with 1 method)

In [2]:
using NetCDF
using NCDatasets

In [3]:
# reanalysis_file_path = "/export/data2/groupMembers/cchristo/reanalysis_3d/merra2/2015/01/MERRA2_400.tavg3_3d_asm_Nv.20150101.nc4"
ave_reanalysis_file_path = "/export/data2/groupMembers/cchristo/cs_matrices/summit_seasonal/profile_djf_week.nc"

"/export/data2/groupMembers/cchristo/cs_matrices/summit_seasonal/profile_djf_week.nc"

In [11]:
# ds = Dataset(ave_reanalysis_file_path)

## Open Profile and Extract Profile for a Time

In [4]:
# ds = ncread(reanalysis_file_path)
ds = Dataset(ave_reanalysis_file_path)

# times = ds["time"][:] 
lats = ds["lat"][:]
lons = ds["lon"][:]
T = ds["T"][:]
P = ds["PL"][:]
# (lon, lat, z, time)
var_array_sizes = size(T)

T_prof = T[:]
P_prof = P[:]

72-element Array{Union{Missing, Float32},1}:
     1.5000001f0
     2.635001f0
     4.014251f0
     5.679252f0
     7.767253f0
    10.452403f0
    13.959901f0
    18.542204f0
    24.493755f0
    32.17835f0
    42.04236f0
    54.62927f0
    70.595665f0
     ⋮
 56360.652f0
 57121.996f0
 57880.805f0
 58639.574f0
 59398.066f0
 60155.59f0
 60910.605f0
 61665.16f0
 62419.117f0
 63172.27f0
 63924.445f0
 64789.258f0

In [5]:
# ncinfo(reanalysis_file_path)
desired_lat = 72.5796
desired_lon = -38.4588

# find index of nearest gridcell
lat_ind = argmin(broadcast(abs, lats .- desired_lat))
lon_ind = argmin(broadcast(abs, lons .- desired_lon))

# set time ind
time_ind = 1

T_prof = T[lat_ind,lon_ind, :,time_ind]
P_prof = P[lat_ind,lon_ind, :,time_ind]

72-element Array{Union{Missing, Float32},1}:
     1.5000001f0
     2.6350012f0
     4.014251f0
     5.679252f0
     7.7672524f0
    10.452404f0
    13.959901f0
    18.542204f0
    24.493753f0
    32.17835f0
    42.04236f0
    54.62927f0
    70.59567f0
     ⋮
 79832.53f0
 81252.28f0
 82670.94f0
 84089.055f0
 85507.18f0
 86924.695f0
 88340.86f0
 89760.266f0
 91180.53f0
 92595.586f0
 94014.875f0
 95438.945f0

In [5]:
# fetch('H2O_S',1,1,xmin,xmax)
# fetch('CO2_S',2,1,xmin,xmax)
# fetch('CH4_S',6,1,xmin,xmax)

In [5]:
hitran_data_CO2 = read_hitran("hitran_molec_id_2_CO2.tar", mol=2, iso=1, ν_min=400, ν_max=2100)
hitran_data_H2O = read_hitran("hitran_molec_id_1_H2O.tar", mol=1, iso=1, ν_min=400, ν_max=2100)
hitran_data_CH4 = read_hitran("hitran_molec_id_6_CH4.tar", mol=6, iso=1, ν_min=400, ν_max=2100)

HitranTable{Float64}
  mol: Array{Int64}((58893,)) [6, 6, 6, 6, 6, 6, 6, 6, 6, 6  …  6, 6, 6, 6, 6, 6, 6, 6, 6, 6]
  iso: Array{Int64}((58893,)) [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
  νᵢ: Array{Float64}((58893,)) [400.416314, 400.500621, 400.515163, 400.545478, 400.783489, 400.847081, 400.941244, 401.101872, 401.494615, 401.738456  …  2098.001522, 2098.224403, 2098.558601, 2099.174328, 2099.274806, 2099.389535, 2099.440549, 2099.731408, 2099.87477, 2099.931237]
  Sᵢ: Array{Float64}((58893,)) [2.055e-30, 2.836e-30, 2.834e-30, 3.576e-30, 6.418e-30, 1.242e-30, 2.422e-30, 1.379e-30, 1.891e-30, 1.795e-30  …  8.137e-28, 1.592e-27, 7.599e-29, 9.206e-29, 3.743e-29, 6.561e-29, 5.98e-29, 7.884e-28, 5.626e-28, 2.94e-29]
  Aᵢ: Array{Float64}((58893,)) [1.727e-7, 1.481e-5, 1.568e-6, 1.997e-7, 6.715e-7, 1.102e-8, 2.125e-8, 1.21e-8, 1.572e-6, 0.001561  …  0.04187, 0.04916, 0.003913, 0.004716, 0.001916, 0.003381, 0.003082, 0.04041, 0.04324, 0.001502]
  γ_air: Array{Float64}(

In [10]:
# model_voigt_CO2 = make_hitran_model(hitran_data_CO2, Voigt(), architecture=Architectures.GPU())


In [7]:
ν_grid = 400:0.01:2100
pressures = 250:250:1250
temperatures = 100:75:400

100:75:400

In [21]:
size(ν_grid)
# size(temperatures)

(170001,)

In [ ]:
@time begin
model_interp = make_interpolation_model(hitran_data_CO2, Voigt(), ν_grid, pressures, temperatures)
    
end

Computing Cross Sections for Interpolation... 20%|█▎    |  ETA: 0:03:14

In [16]:
ν_grid = 400:0.01:2100
p_grid = P_prof[:]
t_grid = T_prof[:]

72-element Array{Union{Missing, Float32},1}:
 202.94925f0
 207.71889f0
 217.35593f0
 226.02596f0
 231.42293f0
 236.14986f0
 241.20515f0
 245.34897f0
 247.8092f0
 250.66423f0
 255.77087f0
 259.24652f0
 263.36966f0
   ⋮
 240.66661f0
 241.12915f0
 241.60477f0
 242.0822f0
 242.54677f0
 242.99382f0
 243.38539f0
 243.70296f0
 244.0438f0
 243.74142f0
 244.11185f0
 237.36955f0

In [19]:
@time begin
CO2_cs_matrix = get_cross_section_matrix(hitran_data_CO2, ν_grid, p_grid, t_grid)
end

Computing Cross Sections for Interpolation...100%|██████| Time: 0:00:06


  6.228824 seconds (1.57 M allocations: 160.072 MiB, 0.65% gc time)


72×170001 Array{Float64,2}:
 4.39053e-33  7.85511e-33  1.84053e-32  …  9.02164e-28  8.76074e-28
 0.0          0.0          0.0             0.0          0.0
 0.0          0.0          0.0             0.0          0.0
 0.0          0.0          0.0             0.0          0.0
 0.0          0.0          0.0             0.0          0.0
 0.0          0.0          0.0          …  0.0          0.0
 0.0          0.0          0.0             0.0          0.0
 0.0          0.0          0.0             0.0          0.0
 0.0          0.0          0.0             0.0          0.0
 0.0          0.0          0.0             0.0          0.0
 0.0          0.0          0.0          …  0.0          0.0
 0.0          0.0          0.0             0.0          0.0
 0.0          0.0          0.0             0.0          0.0
 ⋮                                      ⋱               ⋮
 0.0          0.0          0.0          …  0.0          0.0
 0.0          0.0          0.0             0.0          0.0
 0.0  

In [21]:
CO2_cs_matrix = get_cross_section_matrix(hitran_data_CO2, ν_grid, p_grid, t_grid)
H2O_cs_matrix = get_cross_section_matrix(hitran_data_H2O, ν_grid, p_grid, t_grid)
CH4_cs_matrix = get_cross_section_matrix(hitran_data_CH4, ν_grid, p_grid, t_grid)

Computing Cross Sections for Interpolation...100%|██████| Time: 0:00:06
Computing Cross Sections for Interpolation...100%|██████| Time: 0:00:01
Computing Cross Sections for Interpolation...100%|██████| Time: 0:00:09


72×170001 Array{Float64,2}:
 5.62822e-34  5.64644e-34  5.66513e-34  …  2.94895e-33  2.50534e-33
 0.0          0.0          0.0             0.0          0.0
 0.0          0.0          0.0             0.0          0.0
 0.0          0.0          0.0             0.0          0.0
 0.0          0.0          0.0             0.0          0.0
 0.0          0.0          0.0          …  0.0          0.0
 0.0          0.0          0.0             0.0          0.0
 0.0          0.0          0.0             0.0          0.0
 0.0          0.0          0.0             0.0          0.0
 0.0          0.0          0.0             0.0          0.0
 0.0          0.0          0.0          …  0.0          0.0
 0.0          0.0          0.0             0.0          0.0
 0.0          0.0          0.0             0.0          0.0
 ⋮                                      ⋱               ⋮
 0.0          0.0          0.0          …  0.0          0.0
 0.0          0.0          0.0             0.0          0.0
 0.0  

In [62]:
out_file_path = "/export/data2/groupMembers/cchristo/cs_matrices/summit_seasonal/cs_matrix_djf_week_julia.nc"

"/export/data2/groupMembers/cchristo/cs_matrices/summit_seasonal/cs_matrix_djf_week_julia.nc"

In [76]:
ds = Dataset(out_file_path,"c")

NCDataset: /export/data2/groupMembers/cchristo/cs_matrices/summit_seasonal/cs_matrix_djf_week_julia.nc
Group: /



In [77]:
nu_dim = defDim(ds,"nu", size(ν_grid)[1])
pres_dim = defDim(ds,"pressure", size(p_grid)[1])



# Define the variables temperature with the attribute units
CO2_cs_matrix_var = defVar(ds,"cs_matrix_co2", Float64, ("nu","pressure"))
H2O_cs_matrix_var = defVar(ds, "cs_matrix_h2o", Float64, ("nu","pressure"))
CH4_cs_matrix_var = defVar(ds,"cs_matrix_ch4", Float64, ("nu","pressure"))
nu_coord = defVar(ds,"nu", Float32, ("pressure",))
# pres_coord = defVar(ds,"pressure", Float32, ("nu",))

CO2_cs_matrix_var = CO2_cs_matrix
H2O_cs_matrix_var = H2O_cs_matrix
CH4_cs_matrix_var = CH4_cs_matrix

nu_coord = ν_grid
# pres_coord = p_grid


close(ds)

closed NetCDF NCDataset